## Definitions 

In [148]:
sigs = [
    # metadata
    'name()',
    'symbol()',
    'decimals()',
    # global
    'totalSupply()',
    # transfer related
    'transferFrom(address,address,uint256)',
    'transfer(address, uint256)',
    'balanceOf(address)',
    # approval
    'approve(address,uint256)',
    'allowance(address, address)',
    # deposit
    'deposit()',
    'depositTo(address)',
    'depositAmountTo(address, uint256)',
    'depositAmount(uint256)',
    # withdraw
    'withdraw(uint256)',
    'withdrawTo(address, uint256)',
    'withdrawFrom(address, uint256)',
    'withdrawFromTo(address, address uint256)',
    # permit
    'DOMAIN_SEPARATOR()',
    'nonces(address)',
    'permit(address, address, uint256, uint256, uint8, bytes32, bytes32)',
    # utility
    'multicall(bytes[])',
]

In [157]:
# get Huff define statements
for sig in sigs:
    print(f'#define function {sig} payable returns ()')
    
for sig in sigs:
    fn_name = sig.split('(')[0]
    print(f'{fn_name}_dest:')
    fn_sig = f'__FUNC_SIG({fn_name})'
    print(f'    dup1 {fn_sig:30} sub no_selector_match jumpi')

#define function name() payable returns ()
#define function symbol() payable returns ()
#define function decimals() payable returns ()
#define function totalSupply() payable returns ()
#define function transferFrom(address,address,uint256) payable returns ()
#define function transfer(address, uint256) payable returns ()
#define function balanceOf(address) payable returns ()
#define function approve(address,uint256) payable returns ()
#define function allowance(address, address) payable returns ()
#define function deposit() payable returns ()
#define function depositTo(address) payable returns ()
#define function depositAmountTo(address, uint256) payable returns ()
#define function depositAmount(uint256) payable returns ()
#define function withdraw(uint256) payable returns ()
#define function withdrawTo(address, uint256) payable returns ()
#define function withdrawFrom(address, uint256) payable returns ()
#define function withdrawFromTo(address, address uint256) payable returns ()
#defi

In [34]:
from eth_utils import function_signature_to_4byte_selector

In [35]:
selectors = [
    int.from_bytes(function_signature_to_4byte_selector(sig), 'big')
    for sig in sigs
]
selectors

[117300739,
 2514000705,
 826074471,
 404098525,
 599290589,
 2835717307,
 1889567281,
 157198259,
 3714247998,
 910484757,
 2127478272,
 3573918927,
 2895532248,
 3504541104,
 3076586233,
 2986778176,
 2281220902,
 773487949,
 542910584,
 2490413245,
 3365482902]

## Bit Search
### Bits Required

In [36]:
from math import ceil, log2
bits = int(ceil(log2(len(selectors))))
bits

5

### Basic Search

In [42]:
def mask(n):
    return (1 << n) - 1

In [112]:
def mask_unique(sels, msk):
    masked_ids = set()
    for s in sels:
        masked_id = s & msk
        if masked_id in masked_ids:
            return False
        masked_ids.add(masked_id)
    return True

In [113]:
basic_mask = mask(bits)
basic_offset = None
for i in range(32 - bits + 1):
    if mask_unique(selectors, basic_mask << i):
        basic_offset = i
        break
print(f'basic offset: {basic_offset}')
if basic_offset is not None:
    print(f'mask:{basic_mask << basic_offset:032b}')

basic offset: None


### 1-Gap Search

In [145]:
wmask = None
mul_mask = None
p = 0
for shift in range(0, 32 - bits):
    for gap_size in range(1, 32 - bits - shift + 1):
        for mask_split in range(1, bits):
            p += 1
            mask_low = mask(bits - mask_split)
            mask_hig = mask(mask_split)
            base_smask = (mask_hig << (gap_size + bits - mask_split)) | mask_low
            smask = base_smask << shift
            if mask_unique(selectors, smask):
                wmask = smask
                mul_mask = 1 + (1 << (bits + gap_size))
                break
        if wmask is not None:
            break
    if wmask is not None:
        break
print(f'concluded after {p} iterations')
if wmask is not None:
    
    print(f'base mask: {wmask:032b} [0x{wmask:x}]')
    print(f'mul  mask: {mul_mask:032b} [0x{mul_mask:x}]')
else:
    print(f'mask not found ({p})')

concluded after 1462 iterations
base mask: 00110001110000000000000000000000 [0x31c00000]
mul  mask: 00000000000000000000000100000001 [0x101]


In [142]:
r = wmask * mul_mask
#         00110001110000000000000000000000
print(f'{r:070b}')

0000000000000000000000000000000000000000000000000000100001111100001111
